In [ ]:
!pip install wbdata

!pip install pandas_datareader

In [2]:
import pandas as pd

from pandas_datareader import wb


In [19]:

wb_countries = wb.get_countries()
print("type(wb_countries) ", type(wb_countries))
country_list = wb_countries[wb_countries['region']!='Aggregates']['name'].reset_index(drop=True)
print("type(country_list) ", type(country_list))
print("")
print("wb_countries.columns.tolist()")

print(wb_countries.columns.tolist())
print("")

print (wb_countries)
print (country_list)


type(wb_countries)  <class 'pandas.core.frame.DataFrame'>
type(country_list)  <class 'pandas.core.series.Series'>

wb_countries.columns.tolist()
['iso3c', 'iso2c', 'name', 'region', 'adminregion', 'incomeLevel', 'lendingType', 'capitalCity', 'longitude', 'latitude']

    iso3c iso2c                                               name  \
0     ABW    AW                                              Aruba   
1     AFE    ZH                        Africa Eastern and Southern   
2     AFG    AF                                        Afghanistan   
3     AFR    A9                                             Africa   
4     AFW    ZI                         Africa Western and Central   
..    ...   ...                                                ...   
292   XZN    A5  Sub-Saharan Africa excluding South Africa and ...   
293   YEM    YE                                        Yemen, Rep.   
294   ZAF    ZA                                       South Africa   
295   ZMB    ZM                 

<class 'pandas.core.frame.DataFrame'>


In [4]:
indicator_ids = ['EN.POP.DNST', 'SP.POP.TOTL', 'SM.POP.NETM']

print(indicator_ids)

['EN.POP.DNST', 'SP.POP.TOTL', 'SM.POP.NETM']


In [7]:
migration = wb.WorldBankReader(symbols=['SM.POP.NETM'], start=1950, end=2022, countries='all').read()
pop_density = wb.WorldBankReader(symbols=['EN.POP.DNST'], start=1950, end=2022, countries='all').read()
population = wb.WorldBankReader(symbols=['SP.POP.TOTL'], start=1950, end=2022, countries='all').read()

print(migration)
print(pop_density)
print(population)


                                  SM.POP.NETM
country                     year             
Africa Eastern and Southern 2022          NaN
                            2021    -179444.0
                            2020     -48955.0
                            2019    -187410.0
                            2018    -366105.0
...                                       ...
Zimbabwe                    1964     -10064.0
                            1963      -9369.0
                            1962      -8931.0
                            1961      -8582.0
                            1960      -8256.0

[16758 rows x 1 columns]
                                  EN.POP.DNST
country                     year             
Africa Eastern and Southern 2022          NaN
                            2021          NaN
                            2020    46.150711
                            2019    44.946830
                            2018    43.768985
...                                       ...
Zimbabwe

In [8]:
pop_density_countries = pop_density.reset_index()#.query('country in @country_list').reset_index(drop=True)
population_countries = population.reset_index()#.query('country in @country_list').reset_index(drop=True)
migration_countries = migration.reset_index()#.query('country in @country_list').reset_index(drop=True)

print(pop_density_countries)
print(population_countries)
print(migration_countries)


                           country  year  EN.POP.DNST
0      Africa Eastern and Southern  2022          NaN
1      Africa Eastern and Southern  2021          NaN
2      Africa Eastern and Southern  2020    46.150711
3      Africa Eastern and Southern  2019    44.946830
4      Africa Eastern and Southern  2018    43.768985
...                            ...   ...          ...
16753                     Zimbabwe  1964    11.142127
16754                     Zimbabwe  1963    10.799873
16755                     Zimbabwe  1962    10.468600
16756                     Zimbabwe  1961    10.148512
16757                     Zimbabwe  1960          NaN

[16758 rows x 3 columns]
                           country  year  SP.POP.TOTL
0      Africa Eastern and Southern  2022  720839314.0
1      Africa Eastern and Southern  2021  702977106.0
2      Africa Eastern and Southern  2020  685112979.0
3      Africa Eastern and Southern  2019  667242986.0
4      Africa Eastern and Southern  2018  649757148.0
..

In [9]:

pop_density_merged = pd.merge(population_countries, pop_density_countries, on=['country', 'year'], how='outer')

print(pop_density_merged)

                           country  year  SP.POP.TOTL  EN.POP.DNST
0      Africa Eastern and Southern  2022  720839314.0          NaN
1      Africa Eastern and Southern  2021  702977106.0          NaN
2      Africa Eastern and Southern  2020  685112979.0    46.150711
3      Africa Eastern and Southern  2019  667242986.0    44.946830
4      Africa Eastern and Southern  2018  649757148.0    43.768985
...                            ...   ...          ...          ...
16753                     Zimbabwe  1964    4310332.0    11.142127
16754                     Zimbabwe  1963    4177931.0    10.799873
16755                     Zimbabwe  1962    4049778.0    10.468600
16756                     Zimbabwe  1961    3925952.0    10.148512
16757                     Zimbabwe  1960    3806310.0          NaN

[16758 rows x 4 columns]


In [11]:
migration_final = (pd.merge(pop_density_merged, migration_countries, on=['country', 'year'], how='outer')
                   .rename(columns={'SP.POP.TOTL': 'population',
                                    'EN.POP.DNST': 'pop_density',
                                    'SM.POP.NETM': 'net_migration'})
                   .assign(migration_perc=lambda df: df['net_migration'].div(df['population'])))

column_names = migration_final.columns.tolist()

print(column_names)
print(migration_final)

['country', 'year', 'population', 'pop_density', 'net_migration', 'migration_perc']
                           country  year   population  pop_density  \
0      Africa Eastern and Southern  2022  720839314.0          NaN   
1      Africa Eastern and Southern  2021  702977106.0          NaN   
2      Africa Eastern and Southern  2020  685112979.0    46.150711   
3      Africa Eastern and Southern  2019  667242986.0    44.946830   
4      Africa Eastern and Southern  2018  649757148.0    43.768985   
...                            ...   ...          ...          ...   
16753                     Zimbabwe  1964    4310332.0    11.142127   
16754                     Zimbabwe  1963    4177931.0    10.799873   
16755                     Zimbabwe  1962    4049778.0    10.468600   
16756                     Zimbabwe  1961    3925952.0    10.148512   
16757                     Zimbabwe  1960    3806310.0          NaN   

       net_migration  migration_perc  
0                NaN             NaN

In [12]:
migration_final = pd.merge(migration_final, wb_countries, how='left', left_on='country', right_on='name')

column_names = migration_final.columns.tolist()

print(column_names)
print(migration_final)


['country', 'year', 'population', 'pop_density', 'net_migration', 'migration_perc', 'iso3c', 'iso2c', 'name', 'region', 'adminregion', 'incomeLevel', 'lendingType', 'capitalCity', 'longitude', 'latitude']
                           country  year   population  pop_density  \
0      Africa Eastern and Southern  2022  720839314.0          NaN   
1      Africa Eastern and Southern  2021  702977106.0          NaN   
2      Africa Eastern and Southern  2020  685112979.0    46.150711   
3      Africa Eastern and Southern  2019  667242986.0    44.946830   
4      Africa Eastern and Southern  2018  649757148.0    43.768985   
...                            ...   ...          ...          ...   
16753                     Zimbabwe  1964    4310332.0    11.142127   
16754                     Zimbabwe  1963    4177931.0    10.799873   
16755                     Zimbabwe  1962    4049778.0    10.468600   
16756                     Zimbabwe  1961    3925952.0    10.148512   
16757                    

In [13]:
migration_final.drop('name', axis=1).to_csv('migration_population.csv', index=False)

column_names = migration_final.columns.tolist()

print(column_names)
print(migration_final)


['country', 'year', 'population', 'pop_density', 'net_migration', 'migration_perc', 'iso3c', 'iso2c', 'name', 'region', 'adminregion', 'incomeLevel', 'lendingType', 'capitalCity', 'longitude', 'latitude']
                           country  year   population  pop_density  \
0      Africa Eastern and Southern  2022  720839314.0          NaN   
1      Africa Eastern and Southern  2021  702977106.0          NaN   
2      Africa Eastern and Southern  2020  685112979.0    46.150711   
3      Africa Eastern and Southern  2019  667242986.0    44.946830   
4      Africa Eastern and Southern  2018  649757148.0    43.768985   
...                            ...   ...          ...          ...   
16753                     Zimbabwe  1964    4310332.0    11.142127   
16754                     Zimbabwe  1963    4177931.0    10.799873   
16755                     Zimbabwe  1962    4049778.0    10.468600   
16756                     Zimbabwe  1961    3925952.0    10.148512   
16757                    